In [1]:
from scipy.spatial import cKDTree
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import xgboost
import sklearn
from aif360.sklearn.datasets import fetch_adult,fetch_bank,fetch_compas,fetch_german
from cflib import *

In [2]:
X,y = load_dataset('adult')
df = create_df(X,y,'adult')
df = preprocess(df,'adult')
#grouping instances based on label and prottected attribute
#For example, dataframe of only male(privileged) instances with label 0(affected) and dataframe of only male(privileged) instances with label 1(unaffected)
aff_privileged, aff_unprivileged, unaff_privileged, unaff_unprivileged = group(df,prot_attr='sex') 

In [3]:
aff_privileged

,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Self-emp-not-inc,workclass_Self-emp-inc,workclass_Federal-gov,workclass_Local-gov,...,native-country_Guatemala,native-country_Nicaragua,native-country_Scotland,native-country_Thailand,native-country_Yugoslavia,native-country_El-Salvador,native-country_Trinadad&Tobago,native-country_Peru,native-country_Hong,native-country_Holand-Netherlands
0,25.0,7.0,0.0,0.0,40.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,38.0,9.0,0.0,0.0,50.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,34.0,6.0,0.0,0.0,30.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,55.0,4.0,0.0,0.0,10.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,36.0,13.0,0.0,0.0,40.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20983,32.0,6.0,0.0,0.0,40.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20984,43.0,11.0,0.0,0.0,45.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20985,32.0,14.0,0.0,0.0,11.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20986,22.0,10.0,0.0,0.0,40.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
import numpy as np
import scipy.spatial.distance as dist

def custom_distance(x, y):
    # x and y are arrays of the same length, representing two instances
    # Label encode the categorical features

    # Define the cost for changing each feature
    age_diff = abs(x[0] - y[0])
    age_cost = 100*age_diff

    workclass_diff = 0 if x[1] == y[1] else 1
    workclass_cost = workclass_diff 

    education_diff = abs(x[2] - y[2])
    education_cost = education_diff 

    marital_status_diff = 0 if x[3] == y[3] else 1
    marital_status_cost = marital_status_diff 

    occupation_diff = 0 if x[4] == y[4] else 1
    occupation_cost = occupation_diff 
    
    relationship_diff = 0 if x[5] == y[5] else 1
    relationship_cost = relationship_diff 

    race_diff = 0 if x[6] == y[6] else 1
    race_cost = race_diff * 1000

    capitalgain_diff = abs(x[7] - y[7])
    capitalgain_cost = capitalgain_diff

    capitalloss_diff = abs(x[8] - y[8])
    capitalloss_cost = capitalgain_diff    

    hours_per_week_diff =  abs(x[9] - y[9])
    hours_per_week_cost = hours_per_week_diff
    
    native_country_diff = 0 if x[10] == y[10] else 1
    native_country_cost = native_country_diff 

    # Compute the total distance

    distance = np.sqrt(age_cost**2 + workclass_cost**2 + race_cost**2 + education_cost**2 + marital_status_cost**2 + occupation_cost**2 + relationship_cost**2 + hours_per_week_cost**2 + native_country_cost**2 + capitalgain_cost**2 + capitalloss_cost**2)
    #distance = age_cost + workclass_cost + race_cost + education_cost + marital_status_cost + occupation_cost + relationship_cost + hours_per_week_cost + native_country_cost + capitalgain_cost + capitalloss_cost
    
    return distance

# Compute the distance matrix for the low-income Males and the high-income Males
X1 = aff_privileged.loc[0:200].values
X2 = unaff_privileged.loc[0:200].values
n1 = len(X1)
n2 = len(X2)
distance_matrix = np.zeros((n1, n2))
distance_matrix = np.zeros((n1, n2))
for i,row1 in aff_privileged.loc[0:200].iterrows():
    for j,row2 in unaff_privileged.loc[0:200].iterrows():
        distance_matrix[i, j] = custom_distance(row1.to_numpy(),row2.to_numpy())
#distance_matrix = dist.cdist(X1, X2, metric=custom_distance)


In [15]:
distance_matrix

array([[3.00005000e+02, 7.91930205e+03, 5.00685650e+03, ...,
        1.50000067e+03, 7.81670045e+03, 1.41421356e+00],
       [1.00000200e+03, 7.71137770e+03, 4.10835880e+03, ...,
        2.00007500e+02, 7.45055750e+03, 1.30000077e+03],
       [6.00003333e+02, 7.75276377e+03, 4.36332580e+03, ...,
        6.00002500e+02, 7.54127357e+03, 9.00001111e+02],
       ...,
       [1.70000088e+03, 7.68865053e+03, 3.72405854e+03, ...,
        5.00002000e+02, 7.34171683e+03, 2.00000100e+03],
       [4.00003750e+02, 7.78108906e+03, 4.49873449e+03, ...,
        8.00001250e+02, 7.59412971e+03, 7.00002857e+02],
       [2.20000068e+03, 7.71137763e+03, 3.50978803e+03, ...,
        1.00000100e+03, 7.30416361e+03, 2.50000040e+03]])

In [ ]:
distance_matrix = np.zeros((n1, n2))
for i,row1 in aff_privileged.loc[0:200].iterrows():
    for j,row2 in unaff_privileged.loc[0:200].iterrows():
        distance_matrix[i, j] = custom_distance(row1.to_numpy(),row2.to_numpy())


In [16]:
def normalize_distance_matrix(distance_matrix):
    normalized_distance_matrix = (distance_matrix-np.min(distance_matrix)) / (np.max(distance_matrix)-np.min(distance_matrix))
    return normalized_distance_matrix

normalized_distance_matrix = normalize_distance_matrix(distance_matrix)


In [17]:
k = 1

# Fit the nearest neighbor model to the dataset using the distance matrix
neigh = NearestNeighbors(n_neighbors=k, metric='precomputed')
neigh.fit(normalized_distance_matrix)

# Find the k-nearest neighbors for each instance in the dataset
distances, indices = neigh.kneighbors(normalized_distance_matrix)

In [29]:
aff_privileged.loc[4].compare(unaff_privileged.loc[9])

,self,other
workclass,0.0,1.0
occupation,0.0,9.0


In [9]:
aff_privileged.loc[0]

age               25.0
workclass          2.0
education-num      7.0
marital-status     4.0
occupation         6.0
relationship       3.0
race               2.0
capital-gain       0.0
capital-loss       0.0
hours-per-week    40.0
native-country    38.0
Name: 0, dtype: float64

In [19]:
indexes_priv = pd.DataFrame(indices , columns=['Indexes']) 
indexes_priv['distances'] = distances
indexes_priv

,Indexes,distances
0,200,0.000014
1,193,0.000010
2,6,0.000014
3,17,0.000014
4,9,0.000010
...,...,...
196,38,0.000010
197,64,0.000999
198,165,0.000010
199,104,0.000010


In [11]:
indexes_priv['distances'].value_counts()

0.000003    15
0.000005     9
0.000009     7
0.000030     5
0.000024     4
            ..
0.005315     1
0.000715     1
0.000706     1
0.002821     1
0.000033     1
Name: distances, Length: 127, dtype: int64

In [12]:
unique_priv = indexes_priv['Indexes'].value_counts().index.to_list() 
unique_df_priv = unaff_privileged.loc[unique_priv]

In [13]:
unique_df_priv = unique_df_priv.reset_index()
unique_df_priv = unique_df_priv.drop(columns='index')

In [14]:
# Compute the distance matrix for the low-income Males and the high-income Males
X1 = unique_df_priv.values
X2 = unique_df_priv.values
n1 = len(X1)
n2 = len(X2)
distance_matrix = np.zeros((n1, n2))
for i,row1 in unique_df_priv.iterrows():
    for j,row2 in unique_df_priv.iterrows():
        distance_matrix[i, j] = custom_distance(row1.to_numpy(),row2.to_numpy())

In [15]:
#normalized_distance_matrix = normalize_distance_matrix(distance_matrix)
normalized_distance_matrix = normalize_distance_matrix(distance_matrix)
k = 2

# Fit the nearest neighbor model to the dataset using the distance matrix
neigh = NearestNeighbors(n_neighbors=k, metric='precomputed')
neigh.fit(normalized_distance_matrix)

# Find the k-nearest neighbors for each instance in the dataset
distances, indices = neigh.kneighbors(normalized_distance_matrix)

In [16]:

indexes = []
for i in range(len(indices)):
    indexes.append(indices[i][1])

unique_df_priv['indexes'] = indexes

distance = []

for i in range(len(distances)):
    distance.append(distances[i][1])

unique_df_priv['distances'] = distance

In [17]:
unique_df_priv

,age,workclass,education-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,native-country,indexes,distances
0,23.0,2,9.0,2,11,0,4,0.0,0.0,40.0,38,2,0.000211
1,24.0,2,13.0,2,9,0,4,0.0,0.0,40.0,38,2,0.012195
2,23.0,4,10.0,2,2,0,4,0.0,0.0,40.0,38,0,0.000211
3,34.0,1,13.0,2,2,0,2,0.0,0.0,45.0,38,65,0.048760
4,27.0,2,9.0,2,5,0,4,0.0,0.0,40.0,38,28,0.012190
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,44.0,3,11.0,2,11,0,4,0.0,0.0,45.0,38,57,0.000633
78,57.0,4,10.0,2,3,0,4,0.0,0.0,48.0,38,55,0.000998
79,34.0,2,14.0,2,3,0,4,0.0,1887.0,50.0,38,53,0.000404
80,39.0,0,14.0,4,9,1,4,0.0,0.0,40.0,30,35,0.000559
